<a href="https://colab.research.google.com/github/TasneemBadry/TasneemBadry/blob/main/comp_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import pickle
import sklearn
import pandas as pd
import numpy as np
import holoviews as hv
import nltk
from bokeh.io import output_notebook
output_notebook()

from pathlib import Path

# some seeting for pandas and hvplot

pd.options.display.max_columns = 100
pd.options.display.max_rows = 300
pd.options.display.max_colwidth = 100
np.set_printoptions(threshold=2000)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC,SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from xgboost import XGBClassifier
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
f=pd.read_csv('/content/xy_train.csv')
f

,id,text,label
0,265723,"A group of friends began to volunteer at a homeless shelter after their neighbors protested. ""Se...",0
1,284269,"British Prime Minister @Theresa_May on Nerve Attack on Former Russian Spy: ""The government has c...",0
2,207715,"In 1961, Goodyear released a kit that allows PS2s to be brought to heel. https://m.youtube.com/w...",0
3,551106,"Happy Birthday, Bob Barker! The Price Is Right Host on How He'd Like to Be Remembered | ""As the ...",0
4,8584,"Obama to Nation: 聙""Innocent Cops and Unarmed Young Black Men Should Not be Dying Before Magic Jo...",0
...,...,...,...
59995,70046,"Finish Sniper Simo H盲yh盲 during the invasion of Finland by the USSR (1939, colorized)",0
59996,189377,"Nigerian Prince Scam took $110K from Kansas man; 10 years later, he's getting it back",1
59997,93486,Is It Safe To Smoke Marijuana During Pregnancy? You鈥檇 Be Surprised Of The Answer | no,0
59998,140950,Julius Caesar upon realizing that everyone in the room has a knife except him (44 bc),0


In [ ]:
f.isnull().sum()

id       0
text     0
label    0
dtype: int64

In [ ]:
f.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      60000 non-null  int64 
 1   text    60000 non-null  object
 2   label   60000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.4+ MB


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

stemmer = SnowballStemmer("german")
stop_words = set(stopwords.words("german"))


def clean_text(text, for_embedding=False):
    """ steps:
        - remove any html tags (< /br> often found)
        - Keep only ASCII + European Chars and whitespace, no digits
        - remove single letter chars
        - convert all whitespaces (tabs etc.) to single wspace
        if not for embedding (but e.g. tdf-idf):
        - all lowercase
        - remove stopwords, punctuation and stemm
    """
    RE_WSPACE = re.compile(r"\s+", re.IGNORECASE)
    RE_TAGS = re.compile(r"<[^>]+>")
    RE_ASCII = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE)
    RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE)
    if for_embedding:
        # Keep punctuation
        RE_ASCII = re.compile(r"[^A-Za-zÀ-ž,.!? ]", re.IGNORECASE)
        RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž,.!?]\b", re.IGNORECASE)

    text = re.sub(RE_TAGS, " ", text)
    text = re.sub(RE_ASCII, " ", text)
    text = re.sub(RE_SINGLECHAR, " ", text)
    text = re.sub(RE_WSPACE, " ", text)

    word_tokens = word_tokenize(text)
    words_tokens_lower = [word.lower() for word in word_tokens]

    if for_embedding:
        # no stemming, lowering and punctuation / stop words removal
        words_filtered = word_tokens
    else:
        words_filtered = [
            stemmer.stem(word) for word in words_tokens_lower if word not in stop_words
        ]

    text_clean = " ".join(words_filtered)
    return text_clean

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
%%time
# Clean Comments
f["text_clean"] = f.loc[f["text"].str.len() > 0, "text"]
f["text_clean"] = f["text_clean"].map(
    lambda x: clean_text(x, for_embedding=False) if isinstance(x, str) else x
)

CPU times: user 21.5 s, sys: 49.9 ms, total: 21.5 s
Wall time: 21.6 s


In [ ]:
f

,id,text,label,text_clean
0,265723,"A group of friends began to volunteer at a homeless shelter after their neighbors protested. ""Se...",0,group of friend began to volunte at homeless shelt aft their neighbor protested seeing anoth per...
1,284269,"British Prime Minister @Theresa_May on Nerve Attack on Former Russian Spy: ""The government has c...",0,british prim minist theresa may on nerv attack on form russian spy the government has concluded ...
2,207715,"In 1961, Goodyear released a kit that allows PS2s to be brought to heel. https://m.youtube.com/w...",0,goodyear released kit that allows ps to be brought to heel https youtub com watch alxulk cg zwil...
3,551106,"Happy Birthday, Bob Barker! The Price Is Right Host on How He'd Like to Be Remembered | ""As the ...",0,happy birthday bob bark the pric is right host on how he lik to be remembered as the who said av...
4,8584,"Obama to Nation: 聙""Innocent Cops and Unarmed Young Black Men Should Not be Dying Before Magic Jo...",0,obama to nation innocent cops and unarmed young black men should not be dying befor magic johnso...
...,...,...,...,...
59995,70046,"Finish Sniper Simo H盲yh盲 during the invasion of Finland by the USSR (1939, colorized)",0,finish snip simo yh during the invasion of finland by the ussr colorized
59996,189377,"Nigerian Prince Scam took $110K from Kansas man; 10 years later, he's getting it back",1,nigerian princ scam took from kansas year lat he getting it back
59997,93486,Is It Safe To Smoke Marijuana During Pregnancy? You鈥檇 Be Surprised Of The Answer | no,0,is it saf to smok marijuana during pregnancy you be surprised of the answ no
59998,140950,Julius Caesar upon realizing that everyone in the room has a knife except him (44 bc),0,julius caesar upon realizing that everyon the room has knif except him bc


In [ ]:
# # Create binary grade, class 1-2 or 5-6  = good or bad
# f["grade_bad"] = 0
# f.loc[f["label"] >= 3, "grade_bad"] = np.NaN
# f.loc[f["label"] >= 5, "grade_bad"] = 1

# # Drop when any of x missing
# f = f[(f["text_clean"] != "") & (f["text_clean"] != "null")]

# f = f.dropna(
#     axis="index", subset=["grade_bad", "text", "text_clean"]
# ).reset_index(drop=True)

In [ ]:
f = f.drop(f[f.label ==2].index)

In [ ]:
f.head(10)

,id,text,label,text_clean
0,265723,"A group of friends began to volunteer at a homeless shelter after their neighbors protested. ""Se...",0,group of friend began to volunte at homeless shelt aft their neighbor protested seeing anoth per...
1,284269,"British Prime Minister @Theresa_May on Nerve Attack on Former Russian Spy: ""The government has c...",0,british prim minist theresa may on nerv attack on form russian spy the government has concluded ...
2,207715,"In 1961, Goodyear released a kit that allows PS2s to be brought to heel. https://m.youtube.com/w...",0,goodyear released kit that allows ps to be brought to heel https youtub com watch alxulk cg zwil...
3,551106,"Happy Birthday, Bob Barker! The Price Is Right Host on How He'd Like to Be Remembered | ""As the ...",0,happy birthday bob bark the pric is right host on how he lik to be remembered as the who said av...
4,8584,"Obama to Nation: 聙""Innocent Cops and Unarmed Young Black Men Should Not be Dying Before Magic Jo...",0,obama to nation innocent cops and unarmed young black men should not be dying befor magic johnso...
5,117912,"In the 1920鈥檚, Hitler was forbidden to address public meetings in much of Germany, a major blow ...",0,the hitl forbidd to address public meeting much of germany major blow to the nazis this post by ...
6,213064,"Nerd Wins Scrabble with word you've never heard of | ""Braconid"", meaning 鈥渁ny of numerous wasps ...",0,nerd win scrabbl with word you ve nev heard of braconid meaning ny of numerous wasps of the fami...
7,398923,"Why 95.8% of Female Newscasters Have the Same Hair | ""The style standards are a result of longst...",1,why of femal newscast hav the sam hair the styl standard are result of longstanding requirement ...
8,314798,"Donald Trump Says He'll Do This If More 'Inappropriate' Tapes Come Out | He'll ""continue to talk...",0,donald trump says he ll do this if mor inappropriat tap com out he ll continu to talk about bill...
9,20243,5 crazy facts about Lamborghini's outrageous electric supercar | Only the tires are old; built i...,0,crazy fact about lamborghini outrageous electric supercar only the tir are old built month this ...


In [ ]:
from bokeh.models import NumeralTickFormatter
# Word Frequency of most common words
word_freq = pd.Series(" ".join(f["text_clean"]).split()).value_counts()
word_freq[1:40]

to       30434
of       29139
and      19748
for      12465
it       11354
on       11231
this     10316
is        9991
with      8494
from      7489
my        7291
that      7031
you       6492
his       6391
at        6350
by        5455
he        5208
aft       5125
her       4331
has       4194
as        4163
year      4125
they      3938
are       3786
hav       3518
be        3460
out       3435
post      3383
one       3147
new       3048
not       2999
but       2969
lik       2950
who       2900
their     2855
up        2847
what      2751
can       2749
just      2574
dtype: int64

In [ ]:
# list most uncommon words
word_freq[-10:].reset_index(name="freq")

,index,freq
0,octa,1
1,tempel,1
2,qufu,1
3,tucking,1
4,reappearing,1
5,transnistria,1
6,transnistrian,1
7,gaia,1
8,theia,1
9,wahr,1


In [ ]:
# Distribution of ratings
f["label"].value_counts()

0    32172
1    27596
Name: label, dtype: int64

In [ ]:
"""
Compute unique word vector with frequencies
exclude very uncommon (<10 obsv.) and common (>=30%) words
use pairs of two words (ngram)
"""
vectorizer = TfidfVectorizer(
    analyzer="word", max_df=0.3, min_df=10, ngram_range=(1, 2), norm="l2"
)
vectorizer.fit(f["text_clean"])

TfidfVectorizer(max_df=0.3, min_df=10, ngram_range=(1, 2))

In [ ]:
# Vector representation of vocabulary
word_vector = pd.Series(vectorizer.vocabulary_).sample(5, random_state=1)
print(f"Unique word (ngram) vector extract:\n\n {word_vector}")

Unique word (ngram) vector extract:

 year now       19297
weight loss    18620
on your        11263
duet            4521
cub             3671
dtype: int64


In [ ]:
# Sample data - 25% of data to test set
train, test = train_test_split(f, random_state=1, test_size=0.25, shuffle=True)

X_train = train["text_clean"]
Y_train = train["label"]
X_test = test["text_clean"]
Y_test = test["label"]
print(X_train.shape)
print(X_test.shape)

(44826,)
(14942,)


In [ ]:
# transform each sentence to numeric vector with tf-idf value as elements
X_train_vec = vectorizer.transform(X_train)
X_test_vec = vectorizer.transform(X_test)
X_train_vec.get_shape()

(44826, 19495)

In [ ]:
# Compare original comment text with its numeric vector representation
print(f"Original sentence:\n{X_train[3:4].values}\n")
# Feature Matrix
features = pd.DataFrame(
    X_train_vec[3:4].toarray(), columns=vectorizer.get_feature_names()
)
nonempty_feat = features.loc[:, (features != 0).any(axis=0)]
print(f"Vector representation of sentence:\n {nonempty_feat}")

Original sentence:
['general chuck schum unveiling disastrous suburb invasion plan hillary campaign colorized']

Vector representation of sentence:
    campaign     chuck  colorized  disastrous  general   hillary  invasion  \
0  0.250851  0.371054   0.181547    0.403948  0.26309  0.287688  0.299347   

       plan     schum    suburb  
0  0.231896  0.386448  0.398185  


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# # models to test
# classifiers = [
#     LogisticRegression(solver="sag", random_state=2),
#     # LinearSVC(random_state=1),
#     # RandomForestClassifier(random_state=1),
#     # XGBClassifier(random_state=1),
#     # MLPClassifier(
#     #     random_state=1,
#     #     solver="adam",
#     #     hidden_layer_sizes=(12, 12, 12),
#     #     activation="relu",
#     #     early_stopping=True,
#     #     n_iter_no_change=1,

# ]
# # get names of the objects in list (too lazy for c&p...)
# names = [re.match(r"[^\(]+", name.__str__())[0] for name in classifiers]
# print(f"Classifiers to test: {names}")

In [ ]:
# %%time
# # test all classifiers and save pred. results on test data
# results = {}
# for name, clf in zip(names, classifiers):
#     print(f"Training classifier: {name}")
#     clf.fit(X_train_vec, Y_train)
#     prediction = clf.predict(X_test_vec)
#     report = sklearn.metrics.classification_report(Y_test, prediction)
#     results[name] = report

In [ ]:
# Y_train.value_counts()

In [ ]:
# # Prediction results
# for k, v in results.items():
#     print(f"Results for {k}:")
#     print(f"{v}\n")

In [ ]:
Y_train

57218    1
11585    0
10127    1
36709    0
52642    0
        ..
50254    0
32667    0
5227     0
12258    0
33160    0
Name: label, Length: 44826, dtype: int64

In [ ]:
# feature creation and modelling in a single function
pipe = Pipeline([("tfidf", TfidfVectorizer()), ("logreg", LogisticRegression())])

# define parameter space to test # runtime 35min
params = {
     "tfidf__ngram_range": [(1, 2), (1, 3)],
    "tfidf__max_df": np.arange(0.3, 0.8),
    "tfidf__min_df": np.arange(5, 100),
     #"logreg__C" : np.arange(0.2,0.1,0.15),
     "logreg__penalty" : ['l2','l1']


}
# it is quite slow so we do 4 for now
pipe_SVC_clf = RandomizedSearchCV(pipe, params, n_jobs=-1, scoring="f1_macro", n_iter=3)
pipe_SVC_clf.fit(X_train, Y_train)
pickle.dump(pipe_SVC_clf, open("./clf_pipe.pck", "wb"))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 15.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  Fil

In [ ]:
# print(pipe_clf.best_params_)

In [ ]:
# %%time
# # feature creation and modelling in a single function
# pipe = Pipeline([("tfidf", TfidfVectorizer()), ("svc", LinearSVC())])

# # define parameter space to test # runtime 19min
# params = {
#     "tfidf__ngram_range": [(1, 3)],
#     "tfidf__max_df": [0.5],
#     "tfidf__min_df": [5],
#     "svc__C": np.arange(0.2, 1, 0.15),
# }
# pipe_svc_clf = RandomizedSearchCV(pipe, params, n_jobs=-1, scoring="f1_macro",  n_iter=3)
# pipe_svc_clf.fit(X_train, Y_train)
# pickle.dump(pipe_svc_clf, open("./pipe_svc_clf.pck", "wb"))

In [ ]:
best_params = pipe_SVC_clf.best_params_
print(best_params)

{'tfidf__ngram_range': (1, 3), 'tfidf__min_df': 14, 'tfidf__max_df': 0.3, 'logreg__penalty': 'l2'}


In [ ]:
# run pipe with optimized parameters
pipe.set_params(**best_params).fit(X_train, Y_train)
pipe_pred = pipe_SVC_clf.predict(X_test)
report = sklearn.metrics.classification_report(Y_test, pipe_pred)
print(report)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


              precision    recall  f1-score   support

           0       0.82      0.82      0.82      8035
           1       0.79      0.79      0.79      6907

    accuracy                           0.81     14942
   macro avg       0.81      0.81      0.81     14942
weighted avg       0.81      0.81      0.81     14942



In [ ]:
df=pd.read_csv('/content/x_test.csv')
df

,id,text
0,0,stargazer
1,1,yeah
2,2,PD: Phoenix car thief gets instructions from YouTube video
3,3,"As Trump Accuses Iran, He Has One Problem: His Own Credibility"
4,4,"""Believers"" - Hezbollah 2011"
...,...,...
59146,59146,Bicycle taxi drivers of New Delhi
59147,59147,Trump blows up GOP's formula for winning House races
59148,59148,"Napoleon returns from his exile on the island of Elba. (March 1815), Colourised"
59149,59149,Deep down he always wanted to be a ballet dancer


In [ ]:
df2 = df["text"]

In [ ]:
df2

0                                                                             stargazer 
1                                                                                   yeah
2                             PD: Phoenix car thief gets instructions from YouTube video
3                         As Trump Accuses Iran, He Has One Problem: His Own Credibility
4                                                           "Believers" - Hezbollah 2011
                                              ...                                       
59146                                                  Bicycle taxi drivers of New Delhi
59147                               Trump blows up GOP's formula for winning House races
59148    Napoleon returns from his exile on the island of Elba. (March 1815), Colourised
59149                                   Deep down he always wanted to be a ballet dancer
59150                          Toddler miraculously survives 6-story fall landing on car
Name: text, Length: 5

In [ ]:
submission = pd.DataFrame()

submission['id'] = df.id

submission['label'] = pipe_SVC_clf.predict_proba(df2)[:,1]

submission.to_csv('nlp.csv', index=False)
#this submission is the same best accuracy on kaggle so grid search with any classifier is the best

In [ ]:
%%time
# feature creation and modelling in a single function
pipe = Pipeline([("tfidf", TfidfVectorizer()), ("svc", SVC(kernel='linear',probability=True))])

# define parameter space to test # runtime 19min
params = {
    "tfidf__ngram_range": [(1, 3)],
    "tfidf__max_df": [0.5],
    "tfidf__min_df": [5],
    "svc__C": np.arange(0.2, 1, 0.15),
}
pipe_svc = RandomizedSearchCV(pipe, params, n_jobs=-1, scoring="f1_macro",  n_iter=3)
pipe_svc.fit(X_train, Y_train)
pickle.dump(pipe_svc, open("./pipe_svc_clf.pck", "wb"))

In [ ]:
best_params = pipe_svc.best_params_
print(best_params)

{'tfidf__ngram_range': (1, 3), 'tfidf__min_df': 5, 'tfidf__max_df': 0.5, 'svc__C': 0.35}


In [ ]:
# run pipe with optimized parameters
pipe.set_params(**best_params).fit(X_train, Y_train)
pipe_p = pipe_svc.predict(X_test)
report = sklearn.metrics.classification_report(Y_test, pipe_p)
print(report)

              precision    recall  f1-score   support

           0       0.83      0.82      0.83      8035
           1       0.80      0.80      0.80      6907

    accuracy                           0.81     14942
   macro avg       0.81      0.81      0.81     14942
weighted avg       0.81      0.81      0.81     14942



In [ ]:
submission = pd.DataFrame()

submission['id'] = df.id

submission['label'] = pipe_svc.predict_proba(df2)[:,1]

submission.to_csv('nlp svm.csv', index=False)
#this submission is the same best accuracy on kaggle so grid search with any classifier is the best

AttributeError: ignored